<a href="https://colab.research.google.com/github/kartikeyahl/MS_analysis/blob/main/PunchTimeAnalysis5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from datetime import datetime
import math

In [2]:
# Importing the excel
dataset = pd.read_excel('punch_short_.xlsx',parse_dates=['Date'],
    date_parser=lambda x: pd.to_datetime(x, format='%Y%m%d'))
dataset2=pd.read_excel('Master Data.xlsx')

In [3]:
#Renaming column name to a valid column name format as per python
dataset.rename(columns={'IN/OUT': 'IN_OUT'}, inplace=True)

In [4]:
#Droping rows that conatain 1 in 'IN_OUT' column
dataset=dataset[dataset.IN_OUT!=1]

In [5]:
#Converting Date column values into year-month-day format
# j,k=0,0
# for i in dataset.iloc[:,3]:
#   dataset.Date[j]=datetime.strptime(str(i),'%Y%m%d').date()
#   j+=1

dataset['Date'] = pd.to_datetime(dataset['Date'], errors='coerce')

In [6]:
#Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)
l=[]
for i in range(len(dataset)-1):
  j=i+1
  x=str(dataset.iloc[i]['Time'])
  y=str(dataset.iloc[j]['Time'])

  if len(x)<=5:
    x=x.zfill(6)
  if len(y)<=5:
    y=y.zfill(6)

  x = datetime.strptime(x, "%H%M%S")
  x.strftime("%I:%M %p")
  y = datetime.strptime(y, "%H%M%S")
  y.strftime("%I:%M %p")

  l.append([dataset.iloc[i]['E.Code'], dataset.iloc[i]['Date'].date(), dataset.iloc[i]['IN_OUT'], int(dataset.iloc[i]['Time']), datetime.combine(dataset.iloc[j]['Date'], y.time()) - datetime.combine(dataset.iloc[i]['Date'], x.time())])  
  i=i+2

In [7]:
# #Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)

# x='140000'
# y='020000'
# if len(x)<=4:
#   x=x.zfill(6)
# if len(y)<=4:
#   y=y.zfill(6)

# x2 = datetime.strptime(x, "%H%M%S")
# x2.strftime("%I:%M %p")
# y2 = datetime.strptime(y, "%H%M%S")
# y2.strftime("%I:%M %p")
# print(x2,y2)
# r=datetime.combine(datetime.strptime('20221012','%Y%m%d'), y2.time()) - datetime.combine(datetime.strptime('20221011','%Y%m%d'), x2.time())
# print(r)

In [8]:
#Replacing ":" with "." and getting upto 2 decimal places in Time to make mathematical operations feasible on i
for i in range(len(l)):
  l[i][4] = l[i][4].__str__().replace(":",".")
  s=l[i][4]
  l[i][4]=s[0:-3]

In [9]:
#Removing redundant rows and storing in new data-structure
l2=[]
for i in range(len(l)):
  if 'P20' not in str(l[i][2]):
    l2.append(l[i]) 

In [10]:
for i in range(len(l2)):
  if 'd' in l2[i][4]:
    s=l2[i][4]
    s=float(s[6:len(s)])
    l2[i][4]=24+s

In [11]:
# s='1 day, 0.53'

# s=float(s[5:len(s)])
# s=24+s
# print(s)

In [12]:
#Converting string type to float for Time column
for i in range(len(l2)):
  try:
   l2[i][4]=float("{:.2f}".format(float(l2[i][4])))
  except:
    print(l2[i][0],l2[i][4])

In [13]:
"""Converting time from float to mins, applied logic to check if it is greater than 9hrs & 30mins. Subtacting the 
result to get over time (minutes) converting and storing it into hrs.minutes format. """

l3=[]
for i in range(len(l2)):
  
  frac, whole = math.modf(l2[i][4])
  mins=int(whole*60+frac*100)
  
  if l2[i][2]=='P10' and l2[i][3]>220000:
    # print(frac,whole)
    # print(l2[i][0])
    # print(l2[i][4])
    # print(mins)
    if mins>450:
      mins=mins-450
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      print(ot_time)
      l2[i].append(ot_time)
      l3.append(l2[i])

  else:
    mins=mins-570
    hours=mins//60
    minutes=mins%60
    if minutes<10:
      ot_time = float("{}.0{}".format(hours, minutes))
    else:
      ot_time = float("{}.{}".format(hours, minutes))
    l2[i].append(ot_time)
    l3.append(l2[i])  

0.14
0.02


In [14]:
#Removing redundant rows and storing in new data-structure
l4=[]
for i in range(len(l3)):
  if l3[i][5]>0:
    l4.append(l3[i]) 

In [15]:
# Converting 2d list to dataframe to apply CRUD opertations
df = pd.DataFrame(l4, columns=['E.Code','Date','IN/OUT','Time','Total Time','OT'])

In [16]:
#Droping an unnecesary column
df=df.drop(['IN/OUT','Time','Total Time'], axis=1)

In [17]:
ll1=[]
def binarySearch(arr, l, r, x):

	while l <= r:

		mid = l + (r - l) // 2

		# Check if x is present at mid
		if arr[mid] == x:
			return mid

		# If x is greater, ignore left half
		elif arr[mid] < x:
			l = mid + 1

		# If x is smaller, ignore right half
		else:
			r = mid - 1

	# If we reach here, then the element
	# was not present
	return -1


# Driver Code
arr = [int(i) for i in dataset2['E Code']]

for i in range(len(df)):
  # Function call
  result = binarySearch(arr, 0, len(arr)-1, df.iloc[i]['E.Code'])

  if result != -1:
      ll1.append([dataset2.iloc[result]['Designation'],dataset2.iloc[result]['Operation'],dataset2.iloc[result]['Division'],dataset2.iloc[result]['Department']])
  else:
    print("Element is not present in array")


Element is not present in array


In [18]:
# Converting 2d list to dataframe to apply CRUD opertations
dff = pd.DataFrame(ll1, columns=['Designation','Operation','Division','Department'])

In [19]:
df_final = pd.concat([df['E.Code'], dff, df[['Date','OT']]], axis=1)

In [20]:
""" Printing dataframe (E.Code, Designation, Opr., Division, Dept., Date, OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes)) 
"""
df_final

,E.Code,Designation,Operation,Division,Department,Date,OT
0,32,Chief Manager,General and Corporate Affairs,Industrial Relations,Employee Benefit,2022-10-11,0.14
1,48,General Manager,Customer Service,CS Technical,NaN,2022-10-11,0.03
2,80,Chief Manager,Narsapura Plant,NaN,Projects,2022-10-11,1.53
3,87,General Manager,Production Planning & Control,Production Strategy & Control,NaN,2022-10-11,1.55
4,125,General Manager,Strategic Information System,Manufacturing Operation Systems 4F,NaN,2022-10-11,0.25
...,...,...,...,...,...,...,...
757,30010150,Apprentice,Vithalapur Plant,Production Body Vithalapur,Press & Weld 4F,2022-10-11,0.08
758,30010152,Apprentice,Vithalapur Plant,Production Power Train Vithalapur,AL Machining 4F,2022-10-11,0.08
759,30010162,Apprentice,Vithalapur Plant,Production Power Train Vithalapur,AL Machining 4F,2022-10-11,0.05
760,30010163,Apprentice,Vithalapur Plant,Production Body Vithalapur,Assembly Frame 4F,2022-10-11,0.06


In [21]:
""" Applying Groupby opr. to group dataframe on basis of E.Code and apply sum and count opr. to 
compute Total_OT & Total_OT_Days """

groupby_ecode_Total_OT = df.groupby(['E.Code']).sum()
groupby_ecode_Total_OT_Days = df.groupby(['E.Code']).count()
groupby_operation_sum = df_final.groupby(['Operation']).sum()
groupby_operation = df_final.groupby(['Operation']).count()
groupby_division_sum = df_final.groupby(['Division']).sum()
groupby_division = df_final.groupby(['Division']).count()
groupby_designation_sum = df_final.groupby(['Designation']).sum()
groupby_designation = df_final.groupby(['Designation']).count()

In [22]:
#Converting Groupby object into DataFrame to apply CRUD opr.
df2=pd.DataFrame(groupby_ecode_Total_OT)
df3=pd.DataFrame(groupby_ecode_Total_OT_Days)
df4=pd.DataFrame(groupby_operation)
df5=pd.DataFrame(groupby_division)
df6=pd.DataFrame(groupby_operation_sum)
df7=pd.DataFrame(groupby_division_sum)
df8=pd.DataFrame(groupby_designation)
df9=pd.DataFrame(groupby_designation_sum)

In [23]:
df4=df4.drop(['E.Code','Designation','Division','Department','Date'], axis=1)
df5=df5.drop(['E.Code','Designation','Operation','Department','Date'], axis=1)
df8=df8.drop(['E.Code','Operation','Division','Department','Date'], axis=1)
df6=df6.drop(['E.Code'], axis=1)
df7=df7.drop(['E.Code'], axis=1)
df9=df9.drop(['E.Code'], axis=1)

In [24]:
df4.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
df5.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
df8.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
# df6.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
# df7.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [25]:
l4=[]
for i in df2.index:
      frac, whole = math.modf(df2['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      l4.append(ot_time)

In [26]:
llp=[]
for i in df6.index:
      frac, whole = math.modf(df6['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      llp.append(ot_time)

In [27]:
lllp=[]
for i in df7.index:
      frac, whole = math.modf(df7['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      lllp.append(ot_time)

In [28]:
llllp=[]
for i in df9.index:
      frac, whole = math.modf(df9['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      else:
          ot_time = float("{}.{}".format(hours, minutes))
      llllp.append(ot_time)

In [29]:
df2['Total OT']=l4
df6['Total OT']=llp
df7['Total OT']=lllp
df9['Total OT']=llllp

In [30]:
#Droping an unnecesary column
df3=df3.drop(['Date'], axis=1)

In [31]:
df2=df2.drop(['OT'], axis=1)
df6=df6.drop(['OT'], axis=1)
df7=df7.drop(['OT'], axis=1)
df9=df9.drop(['OT'], axis=1)

In [32]:
#Renaming Columns
#df2.rename(columns={'OT': 'Total OT'}, inplace=True)
df3.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [33]:
#Creating a final dataframe (E.Code, Total OT_Days, Total OT)
result = pd.concat([df3,df2], axis=1)
result_2 =  pd.concat([df4,df6], axis=1)
result_3 =  pd.concat([df5,df7], axis=1)
result_4 =  pd.concat([df8,df9], axis=1)

In [34]:
""" Printing dataframe (E.Code, Total OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes), 
    Total OT_Days) 
"""
result

,Total OT_Days,Total OT
E.Code,,
32,1,0.14
48,1,0.03
80,1,1.53
87,1,1.55
125,1,0.25
...,...,...
30010150,1,0.08
30010152,1,0.08
30010162,1,0.05


In [35]:
# Sorting dataframe in descending order on basis of Total OT_Days (User can get top x employees on basis of Total OT_Days)
result.sort_values(by=['Total OT_Days'], ascending=False)

,Total OT_Days,Total OT
E.Code,,
32,1,0.14
9937,1,4.28
9745,1,2.17
9762,1,0.44
9772,1,0.28
...,...,...
6500,1,3.41
6505,1,1.13
6510,1,1.09


In [36]:
# Sorting dataframe in descending order on basis of Total OT (User can get top x employees on basis of Total OT)
result.sort_values(by=['Total OT'], ascending=False)

,Total OT_Days,Total OT
E.Code,,
1233,1,15.23
9467,1,14.28
30009595,1,9.27
30009834,1,6.05
30010095,1,6.05
...,...,...
2261,1,0.01
6963,1,0.01
7026,1,0.01


In [37]:
result_2.sort_values(by=['Total OT_Days'], ascending=False)

,Total OT_Days,Total OT
Operation,,
Narsapura Plant,267,473.12
Vithalapur Plant,148,178.12
Manesar Plant,76,99.12
Tapukara Plant,70,39.18
Sales & Marketing,41,20.52
Customer Service,40,16.01
Overseas Business,21,38.31
Quality Control Head Office,21,16.52
New Model Purchase & Parts Strategy,17,8.20


In [38]:
result_2.sort_values(by=['Total OT'], ascending=False)

,Total OT_Days,Total OT
Operation,,
Narsapura Plant,267,473.12
Vithalapur Plant,148,178.12
Manesar Plant,76,99.12
Tapukara Plant,70,39.18
Overseas Business,21,38.31
Sales & Marketing,41,20.52
Quality Control Head Office,21,16.52
Customer Service,40,16.01
Production Planning & Control,6,10.25


In [39]:
result_3.sort_values(by=['Total OT_Days'], ascending=False)

,Total OT_Days,Total OT
Division,,
Production Body Narsapura,139,275.08
Production Power Train Narsapura,53,77.29
Production Power Train Vithalapur,44,47.30
Line 3 Vithalapur,43,62.08
Production Body Vithalapur,35,40.50
...,...,...
Production Control Manesar,1,1.45
Manufacturing Operation Systems 1F/2F,1,2.10
Manufacturing Operation Systems 4F,1,0.25


In [40]:
result_3.sort_values(by=['Total OT'], ascending=False)

,Total OT_Days,Total OT
Division,,
Production Body Narsapura,139,275.08
Production Power Train Narsapura,53,77.29
Production Body Manesar,33,68.39
Line 3 Vithalapur,43,62.08
Quality Control Narsapura,24,53.18
...,...,...
Health & Wellness,2,0.10
Accounts,1,0.08
Administration & Security,1,0.05


In [41]:
result_4.sort_values(by=['Total OT_Days'], ascending=False)

,Total OT_Days,Total OT
Designation,,
Manager,173,116.01
Assistant Manager,98,120.59
Apprentice,81,67.50
Chief Manager,80,60.51
Line Associates 1,72,138.05
Line Associates 2,48,150.53
Senior Executive,46,53.14
Fixed Term Associate,41,49.38
Executive,40,36.21


In [42]:
result_4.sort_values(by=['Total OT'], ascending=False)

,Total OT_Days,Total OT
Designation,,
Line Associates 2,48,150.53
Line Associates 1,72,138.05
Assistant Manager,98,120.59
Manager,173,116.01
Apprentice,81,67.50
Staff,20,63.47
Chief Manager,80,60.51
Senior Executive,46,53.14
Fixed Term Associate,41,49.38
